In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
from pathlib import Path
import unidecode
import pandas as pd

import numpy as np
import re
from tqdm.notebook import tqdm
tqdm.pandas()

from utils import BelgeSayisiExtractor, MaddeSayisiExtractor, MevzuatNoExtractor
from eval_utils import filter_dataframe, show_false_predictions, calculate_accuracy
import constants

In [4]:
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 250

In [5]:
df = pd.read_csv("../data/kanunum-nlp-doc-analysis-dataset.csv")

In [6]:
df.shape

(4142, 16)

### Alanlar
- **id**                    : İçerik kimliği
- **kategori**              : İçerik Türü/Kategorisi
- **baslik**                : İçerik Başlığı
- **rega_no**               : İçeriğin yayınlandığı Resmi Gazete Sayısı
- **mukerrer_no**           : İçeriğin yayınlandığı Resmi Gazete Mükerrer Numarası (0 ise mükerrer olmadığını belirtir)
- **rega_tarihi**           : İçeriğin yayınlandığı Resmi Gazete Tarihi (`date:yyyy-mm-dd` formatında)
- **kurum**                 : İçeriği(Genelge, Cumhurbaşkanlığı(CB) Kararnamesi, Tüzük, Yönetmelik, Tebliğ) çıkaran kurum adı
- **mevzuat_no**            : İçeriğin ilgili numarası (Detaylar için ilgili kategoriye bakınız) 
- **belge_sayi**            : Genelge üzerindeki Sayı ifadesinin değeri (sadece işlenebilenler)
- **mevzuat_tarihi**        : İçeriğin ilgili Tarihi (`date:yyyy-mm-dd` formatında)
- **donem**                 : Komisyon Raporunun dönemini (örn: "27. Dönem")
- **sira_no**               : Komisyon Raporunun Sıra Sayısı (örn: "230", "42 ek 1" vb.)
- **madde_sayisi**          : İçeriğin(Kanun, KHK, CB Kararnamesi, Tüzük, Yönetmelik) bölümlerini (Madde sayısını ve işlenemeyen hükümleri/maddeleri, _madde metinlerinin_ içindeki "Madde <sayi> - ..." ifadeleri hariç )
- **data_text**             : İçerik metni (Büyük/Küçük Harf, Noktalama ve "\n" içermektedir)
- **url**                   : İçerik metninin esas alındığı websayfasının linki (kaynak linkler güncel olmayabilir)
- **kanunum_url**           : İçeriğin Kanunum linki

**İçerik Başlıkları**, Kanun, Resmi Gazete, Kanun Hükmünde Kararname, Cumhurbaşkanlığı Kararnamesi, Tebliğ, Yönetmelik ve Tebliğ için içeriğin adı olan başlık kullanılırken Komisyon Raporu, Genelge ve Özelgelerde tanımlayıcı olmak üzere işelenerek üretilmiş bir başlık tercih edilmiştir.

**Kurum Adı**, Kanunum veritabanına işlenmiş hali ile kullanılmaktadır. Örneğin; Üniversite Yönetmeliklerinde kurum adı ayrı ayrı üniversite adı kullanmak yerine tek bir ad ("Üniversite") olarak işlenmiştir. Kurum tespiti yapılırken Bakanlık isimlerinin değişmiş olabileceği ayrıca dikkate alınmalıdır.

## Kategoriler
 Kanun, Resmi Gazete, Kanun Hükmünde Kararname, Cumhurbaşkanlığı Kararnamesi, Genelge, Tebliğ, Yönetmelik, Komisyon Raporu, Tüzük ve Özelge olmak üzere toplam 10 kategori bulunmaktadır.

Tüm içerikler için `kategori`, `baslik`, `data_text`, `url`, `kanunum_url` alanları sunulmuş olup `url` alanı içerik metninin esas alındığı sayfa linkini belirtmektedir. Diğer dikkate alınacak alanlar aşağıdaki gibi olup detaylar alt başlıklarda verilmiştir.


| Kategori                                                               | İçerik Sayısı     | İlgili alanlar (Üst Veriler)                                                                                              |
| :-------                                                               | :------------     | :---------------------------                                                                                              |
| [Kanun](#kategori-kanun)                                               | 514               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url             |
| [Kanun Hükmünde Kararname](#kategori-kanun-hükmünde-kararname-khk)     | 91                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url             |
| [Resmi Gazete](#kategori-resmi-gazete)                                 | 546               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, data_text, url                                                       |
| [Komisyon Raporu](#kategori-komisyon-raporu)                           | 500               | kategori, baslik, donem, sira_no, data_text, url                                                                          |
| [Genelge](#kategori-genelge)                                           | 385               | kategori, baslik, kurum, mevzuat_no, belge_sayi, mevzuat_tarihi, data_text, url                                           |
| [Cumhurbaşkanlığı Kararnamesi](#kategori-cumhurbaşkanlığı-kararnamesi) | 54                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Tüzük](#kategori-tüzük)                                               | 82                | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Yönetmelik](#kategori-yönetmelik)                                     | 597               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, mevzuat_no, mevzuat_tarihi, madde_sayisi, data_text, url      |
| [Tebliğ](#kategori-tebliğ)                                             | 623               | kategori, baslik, rega_no, mukerrer_no, rega_tarihi, kurum, data_text, url                                                |
| [Özelge](#kategori-özelge)                                             | 750               | kategori, baslik, kurum, mevzuat_tarihi, data_text, url M

### BELGE SAYISI

In [7]:
belge_sayisi_extractor = BelgeSayisiExtractor()

In [8]:
belge_sayisi_extractor.extract(df)

'belge_sayi' cikariliyor... -> 'belge_sayi_pred'


  0%|          | 0/385 [00:00<?, ?it/s]

In [9]:
# Çok fazla NaN etiket var
calculate_accuracy(input_df=df,
                   feature_extractor=belge_sayisi_extractor,
                   include_na=True)

Veri Sayisi: 385
Dogru Tahmin Sayisi: 86
Yanlis Tahmin Sayisi: 299
Basari Orani: 0.2234


In [10]:
# NaN olmayan etiketlerde %100 başarı
calculate_accuracy(input_df=df,
                   feature_extractor=belge_sayisi_extractor,
                   include_na=False)

Veri Sayisi: 43
Dogru Tahmin Sayisi: 43
Yanlis Tahmin Sayisi: 0
Basari Orani: 1.0


In [11]:
show_false_predictions(input_df=df,
                       feature_extractor=belge_sayisi_extractor,
                       cols=["baslik", "url", "kategori", "data_text"],
                       include_na=True,
                       first_n=2)

,belge_sayi,belge_sayi_pred,baslik,url,kategori,data_text
917,undefined,B.07.0.GEL.0.44/4412-17-047504,"GELİR İDARESİ BAŞKANLIĞI, GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO:1998/5",http://www.gib.gov.tr/node/89371,Genelge,GELİR VERGİSİ KANUNU İÇ GENELGESİ SIRA NO:1998/47504\n\nTarih 15/12/1998\n\nSayı B.07.0.GEL.0.44/4412-17-047504\n\nKapsam \n\nT.C\n\nMALIYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.44/4412-17-047504 \n\nKONU:GELİR VERGİSİ KANU...
918,undefined,B.07.0.GEL.0.71/7100-89 - 009461,"GELİR İDARESİ BAŞKANLIĞI, VERGİ BARIŞI KANUNU UYGULAMA İÇ GENELGESİ SERİ NO: 2003/2",http://www.gib.gov.tr/node/89248,Genelge,VERGİ BARIŞI KANUNU UYGULAMA İÇ GENELGESİ SERİ NO: 2003/2\n\nTarih 08/03/2003\n\nSayı B.07.0.GEL.0.71/7100-89 - 009461\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelirler Genel Müdürlüğü\n\nSAYI : B.07.0.GEL.0.71/7100-89\n\nTARİH : 08.03.2003\n\nU...


### MADDE SAYISI

In [12]:
madde_sayisi_extractor = MaddeSayisiExtractor()

In [13]:
madde_sayisi_extractor.extract(df)

'madde_sayisi' cikariliyor... -> 'madde_sayisi_pred'


  0%|          | 0/1338 [00:00<?, ?it/s]

/home/nlztrk/Teknofest/notebook/../src/utils.py:81: FutureWarning: Possible nested set at position 23
  ek_gecici_madde_list = [regfound for regfound in re.findall(ek_gecici_madde_regexp, std_txt)]


In [14]:
calculate_accuracy(input_df=df,
                   feature_extractor=madde_sayisi_extractor,
                   include_na=False)

Veri Sayisi: 1338
Dogru Tahmin Sayisi: 1319
Yanlis Tahmin Sayisi: 19
Basari Orani: 0.9858


In [15]:
show_false_predictions(input_df=df,
                       feature_extractor=madde_sayisi_extractor,
                       cols=["baslik", "url", "kategori", "data_text"],
                       include_na=False,
                       first_n=2)

,madde_sayisi,madde_sayisi_pred,baslik,url,kategori,data_text
0,19,20,BÜTÇE KANUNLARINDA YER ALAN BAZI HÜKÜMLERİN İLGİLİ KANUN VE KANUN HÜKMÜNDE KARARNAMELERE EKLENMESİNE DAİR KANUN,https://www.resmigazete.gov.tr/eskiler/2012/07/20120711-1.htm,Kanun,Resmî Gazete Tarihi: 11.07.2012 Resmî Gazete Sayısı: 28350\nKanun No. 6338 Kabul Tarihi : 29/06/2012\nBÜTÇE KANUNLARINDA YER ALAN BAZI HÜKÜMLERİN İLGİLİ KANUN VE KANUN HÜKMÜNDE KARARNAMELERE EKLENMESİNE DAİR KANUN\n\nMadde 1 – 5/1/1961 tarihli ve...
39,44,45,AMME ALACAKLARININ TAHSİL USULÜ HAKKINDA KANUN İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN,https://www.mevzuat.gov.tr/mevzuat?MevzuatNo=6322&MevzuatTur=1&MevzuatTertip=5,Kanun,Resmî Gazete Tarihi: 15.06.2012 Resmî Gazete Sayısı: 28324\nKanun No. 6322 Kabul Tarihi : 31/05/2012\nAMME ALACAKLARININ TAHSİL USULÜ HAKKINDA KANUN İLE BAZI KANUNLARDA DEĞİŞİKLİK YAPILMASINA DAİR KANUN\n\nMadde 1 – 21/7/1953 tarihli ve6183 sayıl...


### MEVZUAT NO

In [16]:
mevzuat_no_extractor = MevzuatNoExtractor()

In [17]:
mevzuat_no_extractor.extract(df)

'mevzuat_no' cikariliyor... -> 'mevzuat_no_pred'


  0%|          | 0/1723 [00:00<?, ?it/s]

In [18]:
calculate_accuracy(input_df=df,
                   feature_extractor=mevzuat_no_extractor,
                   include_na=False)

Veri Sayisi: 1091
Dogru Tahmin Sayisi: 1073
Yanlis Tahmin Sayisi: 18
Basari Orani: 0.9835


In [19]:
show_false_predictions(input_df=df,
                       feature_extractor=mevzuat_no_extractor,
                       cols=["baslik", "url", "kategori", "data_text"],
                       include_na=False,
                       first_n=2)

,mevzuat_no,mevzuat_no_pred,baslik,url,kategori,data_text
1015,2007/ 1,2007/1,"GELİR İDARESİ BAŞKANLIĞI, VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2007/ 1",http://www.gib.gov.tr/node/89380,Genelge,VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2007/ 1\n\nTarih 06/07/2007\n\nSayı B.07.1.GİB.0.05.70/7033-158-60338\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelir İdaresi BaşkanlığI\n\nSayı: B.07.1.GİB.0.05.70/7033-158-60338\n\nVERGİ KİMLİK NUMARA...
1016,2006/ 1,2006/1,"GELİR İDARESİ BAŞKANLIĞI, VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2006/ 1",http://www.gib.gov.tr/node/89369,Genelge,VERGİ KİMLİK NUMARASI İÇ GENELGESİ SERİ NO: 2006/ 1\n\nTarih 31/10/2006\n\nSayı B.07.1.GİB.0.05.70/7033-158-84223\n\nKapsam \n\nT.C.\n\nMALİYE BAKANLIĞI\n\nGelir İdaresi Başkanlığı\n\nSayı : B.07.1.GİB.0.05.70/7033-158-84223\n\nVERGİ KİMLİK NUMAR...


In [20]:
constants.COLS_MEVZUAT_NO

['Kanun',
 'Kanun Hükmünde Kararname',
 'Genelge',
 'Cumhurbaşkanlığı Kararnamesi',
 'Tüzük',
 'Yönetmelik']

In [21]:
filtered_df = filter_dataframe(input_df=df,
                               selected_categories=["Genelge"],
                               custom_filter_array=[df.url.str.contains("kaysis.gov.tr")]
                              )

calculate_accuracy(input_df=filtered_df,
                   feature_extractor=mevzuat_no_extractor,
                   include_na=True)

show_false_predictions(input_df=filtered_df,
                       feature_extractor=mevzuat_no_extractor,
                       cols=["baslik", "url", "kategori", "data_text"],
                       include_na=True,
                       first_n=2)

Veri Sayisi: 154
Dogru Tahmin Sayisi: 142
Yanlis Tahmin Sayisi: 12
Basari Orani: 0.9221


,mevzuat_no,mevzuat_no_pred,baslik,url,kategori,data_text
1139,1046330,2017/1,"SOSYAL GÜVENLİK KURUMU, 2017/1 SAYILI GENELGE",https://kms.kaysis.gov.tr/Home/Goster/105981,Genelge,T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nStrateji Geliştirme Başkanlığı\n\ni rSayı\nU\n\n32945953-010.06.01-E. 1046330 23/02/2017\nKonu-: Strateji Geliştirme Başkanlığı Daire Başkanlıktan \n\nve Alt Birimlerin Görev Tanımlan\ni\n\nİÇ GENELGE \...
1140,2017/07,2017/7,"SOSYAL GÜVENLİK KURUMU, 2017/7 SAYILI GENELGE SİGORTA PRİMLERİNİN ERTELENMESİ",https://kms.kaysis.gov.tr/Home/Goster/96320,Genelge,"T.C.\nSOSYAL GÜVENLİK KURUMU BAŞKANLIĞI \n\nSigorta Primleri Genel Müdürlüğü\n\nSayı : 89075644-202.99-E.695085 07/02/2017\nKonu : Sigorta primlerinin ertelenmesi\n\nGENELGE\n2017-7\n\n18/1/2017 tarihli, 6770 sayılı Türkiye Cumhuriyeti Emekli San..."
